In [4]:
import sqlite3
import pandas as pd
from datetime import datetime
import re
from typing import List, Dict, Any

class BlackoutsDateChecker:
    def __init__(self, db_path: str = "../databases/dataset.db"):
        self.db_path = db_path
        self.conn = None
        self.results = {
            'total_records': 0,
            'empty_start_date': 0,
            'empty_end_date': 0,
            'invalid_start_date': 0,
            'invalid_end_date': 0,
            'end_before_start': 0,
            'future_dates': 0,
            'details': []
        }
    
    def connect(self):
        """Подключение к базе данных"""
        try:
            self.conn = sqlite3.connect(self.db_path)
            print(f"✅ Подключение к {self.db_path} успешно")
            return True
        except Exception as e:
            print(f"❌ Ошибка подключения: {e}")
            return False
    
    def disconnect(self):
        """Закрытие соединения"""
        if self.conn:
            self.conn.close()
            print("🔌 Соединение закрыто")
    
    def get_all_blackouts(self) -> pd.DataFrame:
        """Получение всех записей из таблицы blackouts"""
        query = """
        SELECT id, start_date, end_date, description, type, initiator_name, source
        FROM blackouts
        ORDER BY start_date
        """
        try:
            df = pd.read_sql_query(query, self.conn)
            print(f"📊 Загружено {len(df)} записей")
            return df
        except Exception as e:
            print(f"❌ Ошибка загрузки данных: {e}")
            return pd.DataFrame()
    
    def validate_date_format(self, date_str: str) -> bool:
        """Проверка формата даты (YYYY-MM-DD HH:MM:SS)"""
        if not date_str or pd.isna(date_str):
            return False
        
        # Паттерн для формата 2018-01-01 00:08:00
        pattern = r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$'
        return bool(re.match(pattern, str(date_str)))
    
    def parse_date(self, date_str: str) -> datetime:
        """Парсинг даты в объект datetime"""
        try:
            return datetime.strptime(str(date_str), '%Y-%m-%d %H:%M:%S')
        except ValueError:
            return None
    
    def is_future_date(self, date_obj: datetime) -> bool:
        """Проверка, является ли дата будущей (после сегодняшнего дня)"""
        today = datetime.now()
        return date_obj and date_obj > today
    
    def check_single_record(self, row: pd.Series) -> Dict[str, Any]:
        """Проверка одной записи"""
        issues = []
        start_date_str = str(row['start_date']) if not pd.isna(row['start_date']) else None
        end_date_str = str(row['end_date']) if not pd.isna(row['end_date']) else None
        
        # Проверка пустых дат
        if not start_date_str or start_date_str.lower() in ['none', 'nan', 'null']:
            issues.append("Пустая start_date")
        
        if not end_date_str or end_date_str.lower() in ['none', 'nan', 'null']:
            issues.append("Пустая end_date")
        
        # Проверка формата
        if start_date_str and not self.validate_date_format(start_date_str):
            issues.append(f"Неверный формат start_date: '{start_date_str}'")
        
        if end_date_str and not self.validate_date_format(end_date_str):
            issues.append(f"Неверный формат end_date: '{end_date_str}'")
        
        # Парсинг дат для дальнейших проверок
        start_date = self.parse_date(start_date_str) if start_date_str else None
        end_date = self.parse_date(end_date_str) if end_date_str else None
        
        # Проверка логической последовательности
        if start_date and end_date and end_date < start_date:
            issues.append(f"end_date ({end_date_str}) раньше start_date ({start_date_str})")
        
        # Проверка будущих дат
        if start_date and self.is_future_date(start_date):
            issues.append(f"start_date в будущем: {start_date_str}")
        
        if end_date and self.is_future_date(end_date):
            issues.append(f"end_date в будущем: {end_date_str}")
        
        return {
            'id': row['id'],
            'start_date': start_date_str,
            'end_date': end_date_str,
            'issues': issues,
            'is_valid': len(issues) == 0
        }
    
    def run_full_check(self):
        """Запуск полной проверки"""
        print("\n🚀 НАЧИНАЕМ ПРОВЕРКУ ДАТ В ТАБЛИЦЕ BLACKOUTS\n" + "="*60)
        
        if not self.connect():
            return
        
        df = self.get_all_blackouts()
        if df.empty:
            print("❌ Нет данных для проверки")
            self.disconnect()
            return
        
        self.results['total_records'] = len(df)
        print(f"📈 Всего записей: {len(df)}")
        
        # Проверка каждой записи
        for idx, row in df.iterrows():
            check = self.check_single_record(row)
            self.results['details'].append(check)
            
            if not check['is_valid']:
                if "Пустая start_date" in check['issues']:
                    self.results['empty_start_date'] += 1
                if "Пустая end_date" in check['issues']:
                    self.results['empty_end_date'] += 1
                if "Неверный формат start_date" in check['issues']:
                    self.results['invalid_start_date'] += 1
                if "Неверный формат end_date" in check['issues']:
                    self.results['invalid_end_date'] += 1
                if "end_date раньше start_date" in check['issues']:
                    self.results['end_before_start'] += 1
                if any("в будущем" in issue for issue in check['issues']):
                    self.results['future_dates'] += 1
        
        self.print_summary()
        self.print_detailed_report()
        
        self.disconnect()

    def print_summary(self):
        """Вывод сводки"""
        print("\n📋 СВОДКА ПРОВЕРКИ" + "="*40)
        print(f"✅ Всего записей:          {self.results['total_records']}")
        print(f"✅ Валидных записей:       {len([d for d in self.results['details'] if d['is_valid']])}")
        print(f"❌ Проблемных записей:     {len([d for d in self.results['details'] if not d['is_valid']])}")
        print(f"   • Пустые start_date:    {self.results['empty_start_date']}")
        print(f"   • Пустые end_date:      {self.results['empty_end_date']}")
        print(f"   • Неверный формат start: {self.results['invalid_start_date']}")
        print(f"   • Неверный формат end:  {self.results['invalid_end_date']}")
        print(f"   • end < start:          {self.results['end_before_start']}")
        print(f"   • Будущие даты:         {self.results['future_dates']}")
    
    def print_detailed_report(self):
        """Подробный отчет по проблемным записям"""
        problems = [d for d in self.results['details'] if not d['is_valid']]
        if problems:
            print(f"\n⚠️  ПРОБЛЕМНЫЕ ЗАПИСИ ({len(problems)})" + "="*50)
            for i, problem in enumerate(problems[:20], 1):  # Первые 20
                print(f"\n{i}. ID: {problem['id'][:8]}...")
                print(f"   Start: {problem['start_date']}")
                print(f"   End:   {problem['end_date']}")
                for issue in problem['issues']:
                    print(f"   ❌ {issue}")
            if len(problems) > 20:
                print(f"\n... и еще {len(problems) - 20} проблемных записей")
        else:
            print("\n🎉 ОТЛИЧНО! Все даты корректны!")

# 🚀 ОСНОВНОЙ ЗАПУСК
if __name__ == "__main__":
    checker = BlackoutsDateChecker()
    checker.run_full_check()


🚀 НАЧИНАЕМ ПРОВЕРКУ ДАТ В ТАБЛИЦЕ BLACKOUTS
✅ Подключение к ../databases/dataset.db успешно
📊 Загружено 25264 записей
📈 Всего записей: 25264

📋 СВОДКА ПРОВЕРКИ========================================
✅ Всего записей:          25264
✅ Валидных записей:       25264
❌ Проблемных записей:     0
   • Пустые start_date:    0
   • Пустые end_date:      0
   • Неверный формат start: 0
   • Неверный формат end:  0
   • end < start:          0
   • Будущие даты:         0

🎉 ОТЛИЧНО! Все даты корректны!
🔌 Соединение закрыто
